In [0]:
import tensorflow as tf

In [0]:
layers_in_block = {'DenseNet-121': [6, 12, 24, 16],
                   'DenseNet-169': [6, 12, 32, 32],
                   'DenseNet-201': [6, 12, 48, 32],
                   'DenseNet-265': [6, 12, 64, 48]}

class DenseNet_classification(tf.keras.layers.Layer):
  def __init__(self, pooling='avg', classes=1):
    super(DenseNet_classification, self).__init__()
    self.classes = classes
    self.pooling = pooling

    self.avg_pooling = tf.keras.layers.GlobalAveragePooling2D()
    self.max_pooling = tf.keras.layers.GlobalMaxPooling2D()

    self.sigmoid_fc = tf.keras.layers.Dense(units=classes, activation=tf.keras.activations.sigmoid)
    self.softmax_fc = tf.keras.layers.Dense(units=classes, activation=tf.keras.activations.softmax)


  def call(self, inputs):
    if self.pooling == 'avg':
      x = self.avg_pooling(inputs)
    elif self.pooling == 'max':
      x = self.max_pooling(inputs)

    if self.classes == 1:
      x = self.sigmoid_fc(x)
    else:
      x = self.softmax_fc(x)
      
    return x

def make_dense_layer(pooling='avg', classes=1):
  res_dense = tf.keras.Sequential()
  res_dense.add(DenseNet_classification(pooling=pooling, classes=classes))
  return res_dense

In [0]:
class SEBlock(tf.keras.layers.Layer):
    def __init__(self, input_channels, r=16):
        super(SEBlock, self).__init__()
        self.pool = tf.keras.layers.GlobalAveragePooling2D()
        self.fc1 = tf.keras.layers.Dense(units=input_channels // r)
        self.fc2 = tf.keras.layers.Dense(units=input_channels)

    def call(self, inputs, **kwargs):
        branch = self.pool(inputs)
        branch = self.fc1(branch)
        branch = tf.nn.relu(branch)
        branch = self.fc2(branch)
        branch = tf.nn.sigmoid(branch)
        branch = tf.expand_dims(input=branch, axis=1)
        branch = tf.expand_dims(input=branch, axis=1)
        output = tf.keras.layers.multiply(inputs=[inputs, branch])
        return output

class bottle_building_block(tf.keras.layers.Layer):
  def __init__(self, growth_rate=32, use_se=False):
    self.bn_axis = 3 if tf.keras.backend.image_data_format() == 'channels_last' else 1
    super(bottle_building_block, self).__init__()
    self.conv1 = tf.keras.layers.Conv2D(filters=growth_rate*4,
                                        kernel_size=(1, 1),
                                        use_bias=False)
    self.bn1 = tf.keras.layers.BatchNormalization(axis=self.bn_axis, epsilon=1.001e-5)
    
    self.conv2 = tf.keras.layers.Conv2D(filters=growth_rate,
                                        kernel_size=(3, 3),
                                        padding='same',
                                        use_bias=False)
    self.bn2 = tf.keras.layers.BatchNormalization(axis=self.bn_axis, epsilon=1.001e-5)

    self.use_se = use_se
    self.se_block = SEBlock(input_channels=growth_rate)

  def call(self, inputs, training=None):
    x = self.bn1(inputs, training=training)
    x = tf.nn.relu(x)
    x = self.conv1(x)

    x = self.bn2(x, training=training)
    x = tf.nn.relu(x)
    x = self.conv2(x)
    x = tf.concat([inputs, x], axis=self.bn_axis)

    if self.use_se == True:
        x = self.se_block(x)
        
    return x


class transition_block(tf.keras.layers.Layer):
  def __init__(self, output_channels, reduction):
    super(transition_block, self).__init__()
    self.bn_axis = 3 if tf.keras.backend.image_data_format() == 'channels_last' else 1
    self.output_channels = output_channels

    self.conv1 = tf.keras.layers.Conv2D(filters=int(output_channels),
                                        kernel_size=(1, 1),
                                        use_bias=False)
    self.bn1 = tf.keras.layers.BatchNormalization(axis=self.bn_axis, epsilon=1.001e-5)
    self.avg_pooling = tf.keras.layers.AveragePooling2D(pool_size=(2,2),
                                                        strides=2,
                                                        padding="same")
    
  def call(self, inputs, training=None):
    x = self.bn1(inputs, training=training)
    x = tf.nn.relu(x)
    x = self.conv1(x)
    x = self.avg_pooling(x)

    return x


def dense_transition_block(growth_rate=32, layer_block=None, reduction=0.5, output_channels=None, transition_toggle=True, use_se=False):
  for i in range(layer_block):
    x = bottle_building_block(growth_rate=growth_rate, use_se=use_se)
  if transition_toggle == True:
    x = transition_block(output_channels=output_channels, reduction=reduction)
  return x

class DenseNet_model(tf.keras.Model):
  def __init__(self, num_init_features=64, layer_block='DenseNet-121', \
               growth_rate=32, reduction=0.5, classes=1, pooling='avg', \
               include_top=True, use_se=False):
    super(DenseNet_model, self).__init__()
    self.include_top = include_top

    self.bn_axis = 3 if tf.keras.backend.image_data_format() == 'channels_last' else 1
    
    self.zero_padd_1 = tf.keras.layers.ZeroPadding2D(padding=((3, 3), (3, 3)))
    self.conv_1 = tf.keras.layers.Conv2D(filters=num_init_features, 
                         kernel_size=(7, 7), 
                         strides=2, 
                         use_bias=False)
    self.bn_1 = tf.keras.layers.BatchNormalization(axis=self.bn_axis, epsilon=1.001e-5)
    self.zero_padd_2 = tf.keras.layers.ZeroPadding2D(padding=((1, 1), (1, 1)))
    self.max_pool_1 = tf.keras.layers.MaxPooling2D(pool_size=(3, 3), 
                                                  strides=2)
    self.num_channels = num_init_features
    self.dense_block_1 = dense_transition_block(growth_rate=growth_rate, layer_block=layers_in_block[layer_block][0], 
                                                reduction=reduction, output_channels=self.num_channels, transition_toggle=True, use_se=use_se)
    self.num_channels += growth_rate * layers_in_block[layer_block][0]
    self.num_channels = reduction * self.num_channels
    self.dense_block_2 = dense_transition_block(growth_rate=growth_rate, layer_block=layers_in_block[layer_block][1], 
                                                reduction=reduction, output_channels=self.num_channels, transition_toggle=True, use_se=use_se)
    self.num_channels += growth_rate * layers_in_block[layer_block][1]
    self.num_channels = reduction * self.num_channels
    self.dense_block_3 = dense_transition_block(growth_rate=growth_rate, layer_block=layers_in_block[layer_block][2], 
                                                reduction=reduction, output_channels=self.num_channels, transition_toggle=True, use_se=use_se)
    self.num_channels += growth_rate * layers_in_block[layer_block][2]
    self.num_channels = reduction * self.num_channels
    self.dense_block_4 = dense_transition_block(growth_rate=growth_rate, layer_block=layers_in_block[layer_block][3], 
                                                reduction=reduction, output_channels=self.num_channels, transition_toggle=False, use_se=use_se)
    
    self.bn_2 = tf.keras.layers.BatchNormalization(axis=self.bn_axis, epsilon=1.001e-5)

    self.fc = make_dense_layer(pooling, classes)

  def call(self, inputs, training=None):
    x = self.zero_padd_1(inputs)
    x = self.conv_1(x)
    x = self.bn_1(x, training=training)
    x = tf.nn.relu(x)
    x = self.zero_padd_2(x)
    x = self.max_pool_1(x)

    x = self.dense_block_1(x)
    x = self.dense_block_2(x)
    x = self.dense_block_3(x)
    x = self.dense_block_4(x)

    x = self.bn_2(x)
    x = tf.nn.relu(x)

    if self.include_top == True:
      x = self.fc(x)

    return x

In [0]:
def DenseNet_121():
    return DenseNet_model(num_init_features=64, layer_block='DenseNet-121', growth_rate=32, reduction=0.5, classes=1, pooling='avg')

def DenseNet_169():
    return DenseNet_model(num_init_features=64, layer_block='DenseNet-169', growth_rate=32, reduction=0.5, classes=1, pooling='avg')

def DenseNet_201():
    return DenseNet_model(num_init_features=64, layer_block='DenseNet-201', growth_rate=32, reduction=0.5,classes=1, pooling='avg')

def DenseNet_265():
    return DenseNet_model(num_init_features=64, layer_block='DenseNet-265', growth_rate=32, reduction=0.5,classes=1, pooling='avg')

In [0]:
def se_DenseNet_121():
    return DenseNet_model(num_init_features=64, layer_block='DenseNet-121', growth_rate=32, reduction=0.5, classes=1, pooling='avg', use_se=True)

def se_DenseNet_169():
    return DenseNet_model(num_init_features=64, layer_block='DenseNet-169', growth_rate=32, reduction=0.5, classes=1, pooling='avg', use_se=True)

def se_DenseNet_201():
    return DenseNet_model(num_init_features=64, layer_block='DenseNet-201', growth_rate=32, reduction=0.5,classes=1, pooling='avg', use_se=True)

def se_DenseNet_265():
    return DenseNet_model(num_init_features=64, layer_block='DenseNet-265', growth_rate=32, reduction=0.5,classes=1, pooling='avg', use_se=True)